# Accessing GEDI data using NASA Harmony API  



Harmony provides access to services that can transform data from NASA's [Earth Observing Systems Data and Information System (EOSDIS)](https://www.earthdata.nasa.gov/eosdis) Distributed Active Archive Centers (DAAC). [`harmony-py`](https://github.com/nasa/harmony-py) Python package is the recommended way of interacting with Harmony service. This notebook shows how to query and access customized [NASA's LP DAAC](https://lpdaac.usgs.gov/)  GEDI data outputs using [NASA's Harmony Services](https://harmony.earthdata.nasa.gov/).

## Requirements

- [NASA Earthdata Account](https://urs.earthdata.nasa.gov/home)
- Compatible python environment

We recommend using [mamba](https://mamba.readthedocs.io/en/latest/) to manage Python packages. To install `mamba`, download [miniforge](https://github.com/conda-forge/miniforge) for your operating system. If using Windows, be sure to check the box to "Add mamba to my PATH environment variable" to enable use of mamba directly from your command line interface. **Note that this may cause an issue if you have an existing mamba install through Anaconda.**

You can create a compatible python environment by typing the following into your command line interface:

```
mamba create -n lp_tutorials -c conda-forge --yes python=3.12 fiona gdal hvplot geoviews rioxarray rasterio jupyter geopandas earthaccess jupyter_bokeh h5py h5netcdf spectral scikit-image jupyterlab seaborn dask harmony-py
```

Activate the environment by typing:

```
mamba activate lp_tutorials
```

Launch jupyter notebook by typing:

```
jupyter notebook
```

## Authenticate

In [ ]:
from harmony import BBox, Client, Collection, Request, CapabilitiesRequest
from datetime import datetime
import json
import earthaccess
import geopandas as gp
import os
from IPython.display import JSON
import h5py
import pandas as pd
from shapely.geometry import Point
import hvplot.pandas

os.chdir('../../')

`earthaccess.login()` is used here to access NASA Earthdata Login (EDL) credentials store in a .netrc file and allows users to type their credentials and persist them into a .netrc file if one does not exist. 

In [2]:
auth = earthaccess.login(persist=True)
# auth.token

To access data through Harmony service, we need to create a Harmony Client object using either your EDL token or your Earthdata Login credentials. Below, `username` and `password` are directly provided to `Client` function. See [here](https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb) for other options you can create Harmony Client object. 

In [3]:
harmony_client = Client(auth=(auth.username, auth.password)) 

## Get the GEDI Collections Harmoney Capabilities

Let us start by submitting a capabilities request to see the transformation supported in Harmony API using the GEDI product short names ('GEDI02_A', 'GEDI02_B', 'GEDI01_B'). Besides the `conceptId` and available variables, You can view the supported transformation services. Supported transformation services and variables (such as variable subsetting, bounding box subsetting, shapefile subsetting, concatenation and reprojection, etc.) have `True` value. Based on the returned capabilities information for GEDI , we can submit a variable subsetting request, bounding box/shapefile subsetting, and temporal subsetting.

In [ ]:
capabilities = harmony_client.submit(CapabilitiesRequest(short_name='GEDI02_B'))
print(json.dumps(capabilities, indent=2))

## Create a Harmony Request

There are several parameters that can be used for the Harmoney request. see [the documentation](https://harmony-py.readthedocs.io/en/latest/) and [the introductory Harmony tutorial ](https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb) for more details. 


### GEDI Collection Concept ID

`collection` is a Required parameter. The concept ID, which is the NASA EOSDIS collection ID provided in the Common Metadata Repository (CMR) metadata **OR** Product short name (e.g. 'C2142776747-LPCLOUD') can be provided to search for collections. 

Below, `concept_id` is saved to a variable.  

In [ ]:
print(capabilities['shortName'], ',', capabilities['conceptId'])

concept_id = capabilities['conceptId']


### GEDI Variable Subset

To learn more about the available layers, you can view the GEDI Dictionaries provided in [GEDI products' DOI Landing pages](https://lpdaac.usgs.gov/product_search/?query=gedi&status=Operational&view=cards&sort=title). The available GEDI datasets are also saved into a JSON file (`GEDI_Datasets.json`) stored in `data` folder and is used to view the available datasets here. 



In [ ]:
with open('data/GEDI_Datasets.json', 'r') as fp:
    gedi_var = json.load(fp)

L2B = gedi_var['GEDI_L2B']
L2B[0:25]

In [ ]:
subset_L2B = ['geolocation/lat_lowestmode', 'geolocation/lon_lowestmode', 'geolocation/degrade_flag', 'geolocation/digital_elevation_model', 'geolocation/elev_lowestmode', 'lat_highestreturn', 'geolocation/lon_highestreturn', 'geolocation/elev_highestreturn', 'l2b_quality_flag', 'rh100', 'pai', 'pai_z', 'pavd_z']
# subset_L2B = ['geolocation/lat_lowestmode', 'geolocation/lon_lowestmode', 'rh100']

datasets_p = []
for s in subset_L2B:
    my_var = [v for v in L2B if v.endswith(f'{s}')]
    if len(my_var) == 1:
        datasets_p.append(my_var[0])
        
    elif len(my_var) > 1:
        my_var = [v for v in my_var if v.startswith(f'{s}')]
            
        for l in my_var:
            if l not in datasets_p:
                datasets_p.append(l) 

datasets_p

Select the subset of your desired beams. For instance, you can only select Full Power beams ('BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011').

In [8]:
beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']  #['BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']


In [ ]:
subset = []
for b in beams:
    beam_subset = [f'/{b}/{layer}' for layer in datasets_p]
    [subset.append(i) for i in beam_subset]
subset

### Spatial Subset


Both `spaial` and` shape` are the query parameters used for spatial subsetting using bounding box and shapefile/GeoJSON respectively. For the Bounding box, the Harmony `Bbox` class accepts spatial coordinates as decimal degrees in the order of west, south, east, and north coordinates (e.g.(-119.205104, 36.012018, -117.907297, 37.054834)). For the spatial subset using a region of interest, the path to a GeoJSON file (.json or .geojson), an ESRI Shapefile (.zip or .shz), or a kml file (.kml) as `shape` param are acceptable inputs. 

In [11]:
## path to the GeoJSON
roi = BBox(-119.205104, 36.012018, -117.907297, 37.054834)


### Temporal Subset

For temporal subsetting, the data temporal start and end ranges are used as a datetime object. Below, the start and end dates for the query are selected. 

In [12]:
temporal_range = {'start': datetime(2022, 4, 1), 
                  'stop': datetime(2022, 4, 20)}

Finally, submit the Harmony request and get the request ID.

In [13]:
request = Request(
    collection = Collection(id=concept_id),
    spatial = roi,    # for bbox
    # shape = roi,   # for the GeoJSON
    temporal = temporal_range,
    variables = subset
)

You can check the validity of the request before submitting a Harmony Request.

In [ ]:
request.is_valid()

Finally, submit the request and retrieve list of the processed data URLs once processing is complete. You may optionally show the progress bar below.

In [ ]:
task = harmony_client.submit(request)
print(f'Harmony request ID: {task}')

print(f'Processing your Harmony request:')
task_json = harmony_client.result_json(task, show_progress=True)

Next, download the subset of data. 

In [16]:
results = harmony_client.download_all(task, directory='data', overwrite=True)

In [ ]:
file_names = [f.result() for f in results]

Function is defined to create a dataframe from our HDF5 Harmony subset file. Next, `Geodataframe` is created for all the downloaded subset file. 

In [18]:
def h5_to_dataframe(ds, beams, vars):
    """
    This function takes Harmony subset of GEDI hdf5 and returns a dataframe. 
    """
    #read the dataset
    gedi_ds = h5py.File(ds,'r')
    # see what is the data product 
    product = gedi_ds['METADATA']['DatasetIdentification'].attrs['shortName']
    fileName = gedi_ds['METADATA']['DatasetIdentification'].attrs['fileName']
    date = datetime.strptime(fileName.rsplit('_')[2], '%Y%j%H%M%S').strftime('%Y-%m-%d %H:%M:%S')
    # Create an empty DataFrame for this beam
    df_beam = pd.DataFrame(columns=vars)
    
    for b in beams:
        data_dic = {}
        for v in vars:
            # print(b,v)
            value = gedi_ds[f'{b}/{v}'][()]
            data_dic[v] = value.tolist() 
            
        df_beam = pd.concat([df_beam, pd.DataFrame(data_dic)],join="inner")
        
        # add product, beam, file name, and date columns 
        df_beam.insert(0, 'product', product)
        df_beam.insert(1, 'Beam', b)
        df_beam.insert(2, 'fileName' , fileName)
        df_beam.insert(3, 'date', date)

    return(df_beam.reset_index(drop=True))


In [ ]:
l2b_df = pd.DataFrame()

for file in file_names:
    print(file)
    gedi_subset = h5_to_dataframe(file, beams, datasets_p)
    l2b_df = gp.GeoDataFrame(pd.concat([l2b_df, gedi_subset]))
    del gedi_subset

# Reset the indeces
l2b_df = l2b_df.reset_index(drop=True)
l2b_df = l2b_df.rename(columns={'geolocation/lat_lowestmode': 'lat', 'geolocation/lon_lowestmode': 'lon'})
 # Take the lat/lon from dataframe and convert each lat/lon to a shapely point and convert to a Geodataframe
l2b_df = gp.GeoDataFrame(l2b_df, geometry=l2b_df.apply(lambda row: Point(row.lon, row.lat), axis=1))


In [ ]:
l2b_df.head()

In [ ]:
l2b_df.hvplot(cmap='viridis', clim=(0,l2b_df['pai'].max()), color='pai', size=1, frame_width=900, geo=True, tiles='ESRI')

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 02-20-2024  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  